In [ ]:
# 将当前目录添加到 PYTHONPATH
import sys
import os

cwd = os.path.dirname(os.path.abspath("./server"))
sys.path.append(cwd)
sys.path

In [2]:
import logging
logging.basicConfig(level=logging.INFO)

In [3]:
from client import MCPClient, MCPClientConfig, LLMConfig, MCPServerConfig
from openai_adapter import OpenAIAdapter

In [4]:
config = MCPClientConfig(
    mcp_servers=[
        MCPServerConfig(
            name="music_player",
            transport="sse",
            url="http://localhost:8000/sse"
        )
    ],
    llm=LLMConfig(  
        provider="ollama",
        api_key="ollama",
        base_url="http://localhost:11434/v1",
        sys_prompt="你是一个经验丰富的助手",
    ),
)

In [ ]:
client = MCPClient(config)
await client.connect_to_servers()

In [ ]:
await client.process_query("qwen2.5:latest", "你好")

In [ ]:
async for chunk in client.stream_process_query("qwen2.5:latest", "你好"):
    print(chunk)

In [ ]:
async for chunk in client.stream_process_query("qwen2.5:latest", "你好, 帮我随便放首歌，最好的比较舒缓放松的那种"):
    print(chunk)

In [ ]:
llm_adapter = OpenAIAdapter(
    provider="ollama",
    api_key="ollama",
    base_url="http://localhost:11434/v1",
)

tools, _ = await client.list_all_tools()
print(tools)

In [ ]:
resp = await llm_adapter.chat(
    model="qwen3:latest",
    messages=[
        {"role": "user", "content": "你好, 帮我随便放首歌，最好的比较舒缓放松的那种"}
    ],
    tools=tools,
    stream=True
)
async for chunk in resp:
    print(chunk)



In [ ]:
import ChatTTS
import torch
import torchaudio

chat = ChatTTS.Chat()
chat.load(compile=False) # Set to True for better performance

texts = ["你好，我是小爱同学，很高兴认识你"]

wavs = chat.infer(texts)

for i in range(len(wavs)):
    torchaudio.save(f"basic_output{i}.wav", torch.from_numpy(wavs[i]).unsqueeze(0), 24000)

found invalid characters: {'1'}
found invalid characters: {'2'}
text:   3%|▎         | 10/384(max) [00:01,  5.35it/s]
code:   3%|▎         | 62/2048(max) [00:06,  9.93it/s]
